In [ ]:
import requests, json, logging, time
from notebookutils import mssparkutils

logger = logging.getLogger(__name__)

class fabric_rest:
    def __init__(self, audience:str='pbi'):
        self.header = self.create_header(audience)


    def create_header(self, audience:str='pbi') -> dict:
        return {'Authorization': f'Bearer {mssparkutils.credentials.getToken(audience)}', 'Content-type': 'application/json'}


    def call_rest(self, method:str, url:str, body:dict=None) -> requests.Response():
        try:
            response = requests.request(method=method, url=url, headers=self.header, data=json.dumps(body))
            response.raise_for_status()
            logger.info(f"Success - {response}")
            if response.status_code == 202:
                response = self.response_long_running(response=response)
            return response
        except Exception as e:
            print(f"{e} - {response.text}")


    def response_long_running(self, response:requests.Response()) -> requests.Response():
        responseLocation = response.headers.get('Location')
        for _ in range(5):
            responseStatus = self.call_rest(method='get', url=responseLocation)
            if responseStatus.json().get('status') != 'Succeeded':
                logger.info(f'Operation {response.headers.get("x-ms-operation-id")} is not ready. Waiting for {response.headers.get("Retry-After")} seconds.')
                time.sleep(int(response.headers.get('Retry-After')))
            else:
                logger.info('Payload is ready. Requesting the result.')
                responseResult = self.call_rest(method='get', url=f'{responseLocation}/result')
                return responseResult


    # https://learn.microsoft.com/en-us/rest/api/fabric/admin/workspaces/list-workspaces?tabs=HTTP
    def workspace_get_id(self, workspaceName:str) -> str:
        response = self.call_rest(method='get', url='https://api.fabric.microsoft.com/v1/workspaces')
        workspaceId = [workspace.get('id') for workspace in response.json().get('value') if workspace.get('displayName') == workspaceName][0]
        return workspaceId


    # https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items?tabs=HTTP
    def pipeline_get_id(self, workspaceName:str, pipelineName:str) -> str:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        response = self.call_rest(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items?type=DataPipeline')
        pipelineId = [pipeline.get('id') for pipeline in response.json().get('value') if pipeline.get('displayName') == pipelineName][0]
        return pipelineId


    # https://learn.microsoft.com/en-us/rest/api/fabric/core/items/get-item-definition?tabs=HTTP
    def pipeline_get_item_definition_parts(self, workspaceName:str, pipelineName:str) -> list:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        pipelineId = self.pipeline_get_id(workspaceName=workspaceName, pipelineName=pipelineName)
        response = self.call_rest(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{pipelineId}/getDefinition')
        itemDefinitionParts = response.json().get('definition').get('parts')
        return itemDefinitionParts


    # https://learn.microsoft.com/en-us/rest/api/fabric/core/items/create-item?tabs=HTTP
    def pipeline_clone(self, workspaceNameSource:str, pipelineNameSource:str, workspaceNameTarget:str, pipelineNameTarget:str) -> requests.Response():
        pipelinePartsList = self.pipeline_get_item_definition_parts(workspaceName=workspaceNameSource, pipelineName=pipelineNameSource)

        body = {"displayName": pipelineNameTarget
                ,"type": "DataPipeline"
                ,"definition": {
                    "parts": pipelinePartsList
                    }
                }

        workspaceId = self.workspace_get_id(workspaceName=workspaceNameTarget)
        
        response = self.call_rest(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items', body=body)
        return response

    # https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items?tabs=HTTP
    def notebook_get_id(self, workspaceName:str, notebookName:str) -> str:
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        response = self.call_rest(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items?type=Notebook')
        notebookId = [notebook.get('id') for notebook in response.json().get('value') if notebook.get('displayName') == notebookName][0]
        return notebookId


    def notebook_get_item_definition(self, workspaceName:str, notebookName:str) -> requests.Response():
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        notebookId = self.notebook_get_id(workspaceName=workspaceName, notebookName=notebookName)
        response = self.call_rest(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items/{notebookId}/getDefinition?format=ipynb')
        definition = response.json().get('definition')
        return definition


    def notebook_clone(self, workspaceNameSource:str, notebookNameSource:str, workspaceNameTarget:str, notebookNameTarget:str) -> requests.Response():
        notebook_get_definition = self.notebook_get_item_definition(workspaceName=workspaceNameSource, notebookName=notebookNameSource).get('definition')
        body = {
            "displayName": notebookNameTarget
            ,"type": "Notebook"
            ,"definition": notebook_get_definition
        }
        workspaceIdTarget = self.workspace_get_id(workspaceName=workspaceNameTarget)
        response = self.call_rest(method='post', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceIdTarget}/items', body=body)
        return response
    

    # https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items?tabs=HTTP
    def artifact_response(self, workspaceName:str) -> requests.Response():
        workspaceId = self.workspace_get_id(workspaceName=workspaceName)
        response = self.call_rest(method='get', url=f'https://api.fabric.microsoft.com/v1/workspaces/{workspaceId}/items')
        return response


    def artifact_list(self, workspaceName:str) -> list:
        artifact_response = self.artifact_response(workspaceName=workspaceName)
        artifact_list = artifact_response.json().get('value')
        return artifact_list


    def artifact_get_object(self, workspaceName:str, artifactName:str) -> dict:
        artifact_list = self.artifact_list(workspaceName=workspaceName)
        artifact = [item for item in artifact_list if item.get('displayName') == artifactName][0]
        return artifact


    def artifact_get_id(self, workspaceName:str, artifactName:str) -> str:
        artifactObject = self.artifact_get_object(workspaceName=workspaceName, artifactName=artifactName)
        artifactId = artifactObject.get('id')
        return artifactId


In [ ]:
##
# Unit Tests
##

logging.basicConfig(level=logging.INFO)

# fabric_rest().workspace_get_id(workspaceName='WS_Steve')
# fabric_rest().notebook_get_item_definition(workspaceName='WS_Steve', notebookName='NB_API_ClonePipeline')
# fabric_rest().notebook_clone(workspaceNameSource='WS_Steve', notebookNameSource='NB_API_ClonePipeline', workspaceNameTarget='WS_Bret', notebookNameTarget='NB_API_ClonePipeline_cloned_7')




#### TODO
Create Item - https://learn.microsoft.com/en-us/rest/api/fabric/core/items/create-item \
Delete Item - \
Get Item - https://learn.microsoft.com/en-us/rest/api/fabric/core/items/get-item \
Get Item Definition - https://learn.microsoft.com/en-us/rest/api/fabric/core/items/get-item-definition \
List Items - https://learn.microsoft.com/en-us/rest/api/fabric/core/items/list-items \
Update Item - https://learn.microsoft.com/en-us/rest/api/fabric/core/items/update-item \
Update Item Definition - https://learn.microsoft.com/en-us/rest/api/fabric/core/items/update-item-definition \
Run On Demand Job - https://learn.microsoft.com/en-us/rest/api/fabric/core/job-scheduler/run-on-demand-item-job \
Get Item Job Instance - https://learn.microsoft.com/en-us/rest/api/fabric/core/job-scheduler/get-item-job-instance\
Cancel Item Job Instance - https://learn.microsoft.com/en-us/rest/api/fabric/core/job-scheduler/cancel-item-job-instance \
